In [1]:
import os
import base64
import simplejson as json
import hashlib
import hmac
import httplib2
import time

ACCESS_TOKEN = os.getenv("COINONE_ACCESS_TOKEN")
SECRET_KEY = os.environ.get("COINONE_SECRET_KEY")

In [2]:
URL = 'https://api.coinone.co.kr/v2/account/balance/'
PAYLOAD = {
  "access_token": ACCESS_TOKEN,
}

def get_encoded_payload(payload):
    payload[u'nonce'] = int(time.time()*1000)
    
    dumped_json = json.dumps(payload)
    encoded_json = base64.b64encode(bytes(dumped_json, "utf-8"))
    return encoded_json

def get_signature(encoded_payload, secret_key):
    signature = hmac.new(secret_key.encode('utf-8'), encoded_payload, hashlib.sha512);
    return signature.hexdigest()

def get_response(url, payload):
    encoded_payload = get_encoded_payload(payload)
    headers = {
        'Content-type': 'application/json',
        'X-COINONE-PAYLOAD': encoded_payload,
        'X-COINONE-SIGNATURE': get_signature(encoded_payload, SECRET_KEY)
    }
    http = httplib2.Http()
    response, content = http.request(URL, 'POST', headers=headers, body=encoded_payload)
    return content

def get_result():
    content = get_response(URL, PAYLOAD)
    content = json.loads(content)

    return content

In [3]:
get_result()

{'errorCode': '12', 'result': 'error'}